In [1]:
!pip install opacus

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.8/224.8 kB 4.6 MB/s eta 0:00:00


In [2]:
import torch
import torch.nn as nn
import os
import time
import torch
import argparse
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from torchvision import transforms
from torchvision.datasets import MNIST
from torch.utils.data import DataLoader
from collections import defaultdict
from opacus import PrivacyEngine
import numpy as np

In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [4]:
def idx2onehot(idx, n):

    assert torch.max(idx).item() < n

    if idx.dim() == 1:
        idx = idx.unsqueeze(1)
    onehot = torch.zeros(idx.size(0), n).to(idx.device)
    onehot.scatter_(1, idx, 1)

    return onehot

In [5]:
class VAE(nn.Module):

    def __init__(self, encoder_layer_sizes, latent_size, decoder_layer_sizes,
                 conditional=False, num_labels=0):

        super().__init__()

        if conditional:
            assert num_labels > 0

        assert type(encoder_layer_sizes) == list
        assert type(latent_size) == int
        assert type(decoder_layer_sizes) == list

        self.latent_size = latent_size

        self.encoder = Encoder(
            encoder_layer_sizes, latent_size, conditional, num_labels)
        self.decoder = Decoder(
            decoder_layer_sizes, latent_size, conditional, num_labels)

    def forward(self, x, c=None):

        if x.dim() > 2:
            x = x.view(-1, 28*28)

        means, log_var = self.encoder(x, c)
        z = self.reparameterize(means, log_var)
        recon_x = self.decoder(z, c)

        return recon_x, means, log_var, z

    def reparameterize(self, mu, log_var):

        std = torch.exp(0.5 * log_var)
        eps = torch.randn_like(std)

        return mu + eps * std

    def inference(self, z, c=None):

        recon_x = self.decoder(z, c)

        return recon_x


class Encoder(nn.Module):

    def __init__(self, layer_sizes, latent_size, conditional, num_labels):

        super().__init__()

        self.conditional = conditional
        if self.conditional:
            layer_sizes[0] += num_labels

        self.MLP = nn.Sequential()

        for i, (in_size, out_size) in enumerate(zip(layer_sizes[:-1], layer_sizes[1:])):
            self.MLP.add_module(
                name="L{:d}".format(i), module=nn.Linear(in_size, out_size))
            self.MLP.add_module(name="A{:d}".format(i), module=nn.ReLU())

        self.linear_means = nn.Linear(layer_sizes[-1], latent_size)
        self.linear_log_var = nn.Linear(layer_sizes[-1], latent_size)

    def forward(self, x, c=None):

        if self.conditional:
            c = idx2onehot(c, n=10)
            x = torch.cat((x, c), dim=-1)

        x = self.MLP(x)

        means = self.linear_means(x)
        log_vars = self.linear_log_var(x)

        return means, log_vars


class Decoder(nn.Module):

    def __init__(self, layer_sizes, latent_size, conditional, num_labels):

        super().__init__()

        self.MLP = nn.Sequential()

        self.conditional = conditional
        if self.conditional:
            input_size = latent_size + num_labels
        else:
            input_size = latent_size

        for i, (in_size, out_size) in enumerate(zip([input_size]+layer_sizes[:-1], layer_sizes)):
            self.MLP.add_module(
                name="L{:d}".format(i), module=nn.Linear(in_size, out_size))
            if i+1 < len(layer_sizes):
                self.MLP.add_module(name="A{:d}".format(i), module=nn.ReLU())
            else:
                self.MLP.add_module(name="sigmoid", module=nn.Sigmoid())

    def forward(self, z, c):

        if self.conditional:
            c = idx2onehot(c, n=10)
            z = torch.cat((z, c), dim=-1)

        x = self.MLP(z)

        return x


In [6]:
seed=0
batch_size=128
encoder_layer_sizes = [784, 256]
latent_size = 2
decoder_layer_sizes = [256, 784]
conditional = True
learning_rate = 0.01
epochs = 300
print_every = 500
fig_root = "figs"

In [7]:
torch.manual_seed(seed)
if torch.cuda.is_available():
    torch.cuda.manual_seed(seed)

def loss_fn(recon_x, x, mean, log_var):
    BCE = torch.nn.functional.binary_cross_entropy(
        recon_x.view(-1, 28*28), x.view(-1, 28*28), reduction='sum')
    KLD = -0.5 * torch.sum(1 + log_var - mean.pow(2) - log_var.exp())

    return (BCE + KLD) / x.size(0)


In [8]:
vae = torch.load("/kaggle/input/models/cVAE-pytorch.pth")

directory = "c_base_img_samples"
if not os.path.exists(directory):
    os.makedirs(directory)
    
for i in range(10):
    for j in range(600):
    # test with new model using copied tensor
        print(i,j)
        c = torch.arange(i,i+1).long().unsqueeze(1).to(device)
        z = torch.randn([c.size(0), latent_size]).to(device)
        x_hat = vae.inference(z, c=c)
        outpt = x_hat[0].reshape(28, 28).to('cpu').detach().numpy()
        if not os.path.exists(directory+"/"+str(i)):
            os.makedirs(directory+"/"+str(i))
        plt.imsave(f'/kaggle/working/c_base_img_samples/{i}/{j}.png',np.array(outpt), cmap='gray')

0 0
0 1
0 2
0 3
0 4
0 5
0 6
0 7
0 8
0 9
0 10
0 11
0 12
0 13
0 14
0 15
0 16
0 17
0 18
0 19
0 20
0 21
0 22
0 23
0 24
0 25
0 26
0 27
0 28
0 29
0 30
0 31
0 32
0 33
0 34
0 35
0 36
0 37
0 38
0 39
0 40
0 41
0 42
0 43
0 44
0 45
0 46
0 47
0 48
0 49
0 50
0 51
0 52
0 53
0 54
0 55
0 56
0 57
0 58
0 59
0 60
0 61
0 62
0 63
0 64
0 65
0 66
0 67
0 68
0 69
0 70
0 71
0 72
0 73
0 74
0 75
0 76
0 77
0 78
0 79
0 80
0 81
0 82
0 83
0 84
0 85
0 86
0 87
0 88
0 89
0 90
0 91
0 92
0 93
0 94
0 95
0 96
0 97
0 98
0 99
0 100
0 101
0 102
0 103
0 104
0 105
0 106
0 107
0 108
0 109
0 110
0 111
0 112
0 113
0 114
0 115
0 116
0 117
0 118
0 119
0 120
0 121
0 122
0 123
0 124
0 125
0 126
0 127
0 128
0 129
0 130
0 131
0 132
0 133
0 134
0 135
0 136
0 137
0 138
0 139
0 140
0 141
0 142
0 143
0 144
0 145
0 146
0 147
0 148
0 149
0 150
0 151
0 152
0 153
0 154
0 155
0 156
0 157
0 158
0 159
0 160
0 161
0 162
0 163
0 164
0 165
0 166
0 167
0 168
0 169
0 170
0 171
0 172
0 173
0 174
0 175
0 176
0 177
0 178
0 179
0 180
0 181
0 182
0 183
0 184


In [9]:
!zip -r base_cvae_imgs.zip /kaggle/working/c_base_img_samples/

  adding: kaggle/working/c_base_img_samples/ (stored 0%)
  adding: kaggle/working/c_base_img_samples/9/ (stored 0%)
  adding: kaggle/working/c_base_img_samples/9/128.png (stored 0%)
  adding: kaggle/working/c_base_img_samples/9/170.png (stored 0%)
  adding: kaggle/working/c_base_img_samples/9/23.png (stored 0%)
  adding: kaggle/working/c_base_img_samples/9/530.png (stored 0%)
  adding: kaggle/working/c_base_img_samples/9/154.png (stored 0%)
  adding: kaggle/working/c_base_img_samples/9/107.png (deflated 1%)
  adding: kaggle/working/c_base_img_samples/9/336.png (stored 0%)
  adding: kaggle/working/c_base_img_samples/9/145.png (deflated 1%)
  adding: kaggle/working/c_base_img_samples/9/21.png (deflated 1%)
  adding: kaggle/working/c_base_img_samples/9/532.png (stored 0%)
  adding: kaggle/working/c_base_img_samples/9/283.png (stored 0%)
  adding: kaggle/working/c_base_img_samples/9/208.png (stored 0%)
  adding: kaggle/working/c_base_img_samples/9/290.png (stored 0%)
  adding: kaggle/worki

DP Models

In [10]:
vae = torch.load("/kaggle/input/models/cVAE-DP-pytorch_40.pth")

directory = "dp40_img_samples"
if not os.path.exists(directory):
    os.makedirs(directory)
    
for i in range(10):
    for j in range(600):
    # test with new model using copied tensor
        print(i,j)
        c = torch.arange(i,i+1).long().unsqueeze(1).to(device)
        z = torch.randn([c.size(0), latent_size]).to(device)
        x_hat = vae.inference(z, c=c)
        outpt = x_hat[0].reshape(28, 28).to('cpu').detach().numpy()
        if not os.path.exists(directory+"/"+str(i)):
            os.makedirs(directory+"/"+str(i))
        plt.imsave(f'/kaggle/working/dp40_img_samples/{i}/{j}.png',np.array(outpt), cmap='gray')

RuntimeError: Attempting to deserialize object on a CUDA device but torch.cuda.is_available() is False. If you are running on a CPU-only machine, please use torch.load with map_location=torch.device('cpu') to map your storages to the CPU.

In [ ]:
!zip -r dp40_cvae_imgs.zip /kaggle/working/dp40_img_samples/